In [28]:
import xml.etree.ElementTree as ET
import re

# Load the XML file
xml_file_path = 'haarcascade_frontalface_alt.xml'
tree = ET.parse(xml_file_path)
root = tree.getroot()

# Define the original float range and target integer range
FLOAT_MIN = -1.0  
FLOAT_MAX = 106.0   
TARGET_MIN = 0
TARGET_MAX = 65536

# Function to quantize a float to the 0-255 range
def quantize_float(value):
    float_value = float(value)
    # Scale float_value to the TARGET_MIN - TARGET_MAX range
    scaled_value = (float_value - FLOAT_MIN) / (FLOAT_MAX - FLOAT_MIN) * (TARGET_MAX - TARGET_MIN) + TARGET_MIN
    quantized_value = round(scaled_value)  # Round to the nearest integer
    return str(int(quantized_value))

# Function to process each text or attribute for float values
def process_text_for_floats(text):
    return ' '.join(
        quantize_float(num) if re.match(r"-?\d+\.\d+(e[+-]?\d+)?", num) else num
        for num in text.split()
    )

# Traverse all elements to apply quantization to floats in both text and attributes
for elem in root.iter():
    # Quantize floats in element text
    if elem.text and any(re.match(r"-?\d+\.\d+(e[+-]?\d+)?", num) for num in elem.text.split()):
        elem.text = process_text_for_floats(elem.text)
    
    # Quantize floats in element attributes
    for attr, value in elem.attrib.items():
        if re.match(r"-?\d+\.\d+(e[+-]?\d+)?", value):
            elem.attrib[attr] = quantize_float(value)

# Save the modified XML
output_file_path = 'quantized_haarcascade_frontalface_alt.xml'
tree.write(output_file_path)

print(f"Quantized XML saved to: {output_file_path}")


Quantized XML saved to: quantized_haarcascade_frontalface_alt.xml


In [29]:
import xml.etree.ElementTree as ET
import re

# Load the XML file with quantized integers
xml_file_path = 'quantized_haarcascade_frontalface_alt.xml'
tree = ET.parse(xml_file_path)
root = tree.getroot()

# Define the original float range and target integer range
FLOAT_MIN = -1.0  # Minimum expected float value
FLOAT_MAX = 106.0   # Maximum expected float value
TARGET_MIN = 0
TARGET_MAX = 65536

# Function to convert quantized integer back to float
def dequantize_int(value):
    int_value = int(value)
    # Reverse the scaling from TARGET_MIN - TARGET_MAX back to FLOAT_MIN - FLOAT_MAX
    original_float = (int_value - TARGET_MIN) / (TARGET_MAX - TARGET_MIN) * (FLOAT_MAX - FLOAT_MIN) + FLOAT_MIN
    return f"{original_float:.8f}"  # Format to maintain float precision

# Function to process each text or attribute for integer values
def process_text_for_integers(text):
    return ' '.join(
        dequantize_int(num) if num.isdigit() else num
        for num in text.split()
    )

# Traverse all elements to apply dequantization to integers in both text and attributes
for elem in root.iter():
    # Dequantize integers in element text
    if elem.text and all(num.isdigit() for num in elem.text.split()):
        elem.text = process_text_for_integers(elem.text)
    
    # Dequantize integers in element attributes
    for attr, value in elem.attrib.items():
        if value.isdigit():
            elem.attrib[attr] = dequantize_int(value)

# Save the modified XML
output_file_path = 'dequantized_haarcascade_frontalface_alt.xml'
tree.write(output_file_path)

print(f"Dequantized XML saved to: {output_file_path}")


Dequantized XML saved to: dequantized_haarcascade_frontalface_alt.xml
